# Graph build

In [64]:
# #preprocessing
# with open('gold_label') as f:
#     gold_labels = f.readlines()
# gold_questions = [gold_labels[2*i] for i in range(200)]
# gold_questions = list(map(lambda x: x.replace('\t',' '), gold_questions))
# print (gold_questions)
# with open('gold_question','w') as f:
#     for question in gold_questions:
#         f.write(question)

In [4]:
import json

with open('../data/DM.sdp') as f:
    corpus = f.read()
    
with open('../data/gold_graph') as f:
    gold_graphs = json.load(f)[100:]
    
with open('../data/bert_graph') as f:
    bert_graphs = json.load(f)
    
# with open('../data/gold_label') as f:
#     gold_labels = f.read()[200:]

# with open('../data/result/bert_label') as f:
#     bert_labels = f.read()

In [5]:
dms = corpus.split('\n\n')[100:]

In [6]:
def dm_to_graph(dm,nodes):
    # build word to node map
    anchor = []
    for row in dm:
        if row[5] == '+':
            anchor.append(row[0])

    # create all_edges without heuristic
    all_edges = set()
    other_edges = set()
    for row in dm:
        arcs = row[7:]
        for arc_num in range(len(arcs)):
            node_from = dm[int(anchor[arc_num])-1][2]
            node_to = row[2]
            word_from = dm[int(anchor[arc_num])-1]
            word_to = row
            # word-level edges
            if arcs[arc_num] != '_':
                if node_from == node_to:
                    continue
                elif type(node_from)==int and type(node_to)==int:
                    all_edges.add((node_from, node_to, arcs[arc_num]))
                # at least one word is not a node
                else:
                    other_edges.add((node_from, node_to, arcs[arc_num]))
                    #print (node_from, node_to)
    # uncomment to disable heuristic
    # return all_edges.union(other_edges)
    
    # heuristic begin
    # ------------------------------------------------------------#
    # if some nodes are connected by other_edges, add it,
    for edge in other_edges:
        for e in other_edges:
            if edge != e and edge[0] == e[0] and type(edge[1])==int and type(e[1])==int:
                if edge[2] == "ARG1" or e[2] == 'ARG2':
                    all_edges.add((edge[1],e[1],''))
                else:
                    all_edges.add((e[1],edge[1],''))
            if edge != e and edge[0] == e[1] and type(edge[1])==int and type(e[0])==int:
                    all_edges.add((e[0],edge[1],''))
    
    # modify all_edges use some rule
    dm_edges = set()
    
    # step 1 wh-word
    flag=False
    for edge in list(all_edges):
        # if edge from wh-word, add it
        if nodes[edge[0]]['code'] == 'num' or nodes[edge[0]]['code'].isdigit():
            flag = True
            # if the edge label is 'loc', find the anchor, merge it
            if edge[2] == 'loc':
                for e in list(all_edges):
                    if e[0] == edge[1] and e[2]=='ARG2':
                        all_edges.remove(e)
                        all_edges.remove(edge)
                        edge = (edge[0],e[1],'ARG1')
                        dm_edges.add(edge)
            else:
                all_edges.remove(edge)
                dm_edges.add(edge)
    if not flag:
        for i,node in enumerate(nodes):
            if node['code'] == 'num' or node['code'].isdigit():
                # check other node, whether some nodes link wh-word with another node
                for edge in other_edges:
                    if edge[1]==i and edge[2]=='ARG1':
                        for e in other_edges:
                            if e[0] == edge[0] and e[2] == 'ARG2':
                                flag = True
                                dm_edges.add((edge[1],e[1],edge[0]))
                # else add wh-word with its nearest node
                if not flag:    
                    dm_edges.add((i,i+1,'ARG1'))
    for edge in list(dm_edges):
        if (nodes[edge[0]]['code'] == 'num' or nodes[edge[0]]['code'].isdigit()) and nodes[edge[0]]['text'][:2] in ['Is', 'Do', 'Ar']:
            dm_edges.remove(edge)
        
                    
    for edge in list(all_edges):
        if (nodes[edge[0]]['code'] == 'num' or nodes[edge[0]]['code'].isdigit()) and nodes[edge[0]]['text'][:2] in ['Is', 'Do', 'Ar']:
            all_edges.remove(edge)
        # if edge is from r to n or t, add it
        if nodes[edge[0]]['code'] == 'r' and nodes[edge[1]]['code'] in ['n','t']:
            all_edges.remove(edge)
            dm_edges.add(edge)
            
    for edge in list(all_edges):
        # edge from r to d, check the label (ARG1 should be refine)
        if nodes[edge[0]]['code'] == 'r' and nodes[edge[1]]['code'] == 'd' and edge[2]=='ARG1':
            flag = True
            for e in dm_edges:
                if e == (e[0],_,'ARG2'):
                    flag = True
                    break
            if flag:
                all_edges.remove(edge)
                for i in range(edge[0],0,-1):
                    if nodes[i]['code'] in ['t','n']:
                        dm_edges.add((edge[0], i, edge[2]))

    for edge in list(all_edges):
        # edge from r to d, check the label (ARG2 is ok, but check if ARG1 exist)
        if nodes[edge[0]]['code'] == 'r' and nodes[edge[1]]['code'] == 'd' and edge[2]=='ARG2':
            all_edges.remove(edge)
            dm_edges.add(edge)
            for e in list(dm_edges):
                # remove the wrong label
                if e[0]==edge[0] and e[2]!='ARG1':
                    dm_edges.remove(e)
                    break
            for e1 in list(dm_edges):
                for e2 in list(dm_edges):
                    if e1[0]==e2[0] and e1[2]=='ARG1' and e2[2]=='ARG2':
                        dm_edges.add((edge[0],e1[0],'ARG1'))
                        
    # heuristic end
    # ------------------------------------------------------------#
    
    #     print (f'dm_edges: {dm_edges}')
    #     print (f'remain: {all_edges}')

    return dm_edges.union(all_edges)

In [7]:
def to_true_edge(nodes,set1):
    set1 = set(list(map(lambda x: (nodes[x[0]]['text'], nodes[x[1]]['text'], x[2]) if type(x[0])==int and type(x[1])==int else x, list(set1))))
    return set1

y,y_pred = [],[]
for dm,gold_graph,bert_graph in zip(dms, gold_graphs, bert_graphs):
    dm = list(filter(lambda x: x and x[0]!='#', dm.split('\n')))
    dm = list(map(lambda x: x.split('\t'), dm))
    
    gold_nodes = gold_graph['nodes']
    bert_nodes = bert_graph['nodes']
    sentence = gold_graph['sentence']
    # build gold edges
    gold_edges = set()
    for node_num,node in enumerate(bert_nodes):
        # assign dm[2] to node_num
        for i in range(node['start'],node['end']):
            dm[i][2] = node_num
    for node_num,node in enumerate(gold_nodes):
        if node['arg1'][0]!=-1:
            gold_edges.add((node_num,node['arg1'][0],'ARG1'))
        if node['arg2'][0]!=-1:
            gold_edges.add((node_num,node['arg2'][0],'ARG2'))

    assert len(sentence.split()) == len(dm)
    pred_edges = dm_to_graph(dm, bert_nodes)
    y.append(to_true_edge(gold_nodes,gold_edges))
    y_pred.append(to_true_edge(bert_nodes,pred_edges))

# Graph Eval

In [8]:
def same_graph(set1,set2):
    set1 = set(list(map(lambda x: (x[0],x[1]), set1)))
    set2 = set(list(map(lambda x: (x[0],x[1]), set2)))
    return set1==set2

def edge_stats(set1,set2):
    set1 = set(list(map(lambda x: (x[0],x[1]), set1)))
    set2 = set(list(map(lambda x: (x[0],x[1]), set2)))
    return len(set1.intersection(set2)), len(set1), len(set2)
    
def eval(y,y_pred):
    # edge level
    tp, t, p = 0, 0, 0
    for yy,yy_pred in zip(y,y_pred):
        tp1, t1, p1 = edge_stats(yy,yy_pred)
        tp += tp1
        t += t1
        p += p1
    precision = tp/p
    recall = tp/t
    f1 = 2*precision*recall/(precision+recall)
    print(f'Edge_level, p:{100*precision}, r:{100*recall}, f1:{100*f1}')
    # graph level
    tp = 0
    for (yy,yy_pred) in zip(y,y_pred):
        tp += same_graph(yy,yy_pred)
    print(f'Graph_level acc: {100*tp/len(y)}')
    

In [12]:
eval(y,y_pred)

Edge_level, p:69.92084432717678, r:71.62162162162163, f1:70.76101468624833
Graph_level acc: 44.0


In [9]:
# build random baseline
import random
y_pred = []
for gold_graph in gold_graphs:
    gold_nodes = gold_graph['nodes']
    pred = []
    for st in range(len(gold_nodes)):
        for ed in range(len(gold_nodes)):
            if st!=ed:
                if random.randint(0,1)==1:
                    pred.append((st,ed,'RD'))
    y_pred.append(to_true_edge(gold_nodes, pred))

In [10]:
eval(y,y_pred)

Edge_level, p:15.067340067340067, r:48.37837837837838, f1:22.978177150192554
Graph_level acc: 0.0


In [13]:
# pipelined result (integrate manual evaluation)
final_acc = 0
with open('../data/mannual_evaluation/final.txt') as f:
    manual = f.readlines()
for yy, yy_pred, m in zip(y,y_pred,manual):
    if same_graph(yy,yy_pred) and m.strip() == 'True':
        final_acc += 1
print(final_acc)

38


In [153]:
# build dep_baseline
from allennlp.predictors.predictor import Predictor
dep_predictor = Predictor.from_path("/home/haonanl5/tools/allennlp_model/biaffine-dependency-parser-ptb-2018.08.23.tar.gz")

Did not use initialization regex that was passed: .*bias_ih.*
Did not use initialization regex that was passed: .*bias_hh.*
Did not use initialization regex that was passed: .*weight_hh.*
Did not use initialization regex that was passed: .*weight_ih.*


In [2]:
y,y_pred = [],[]
for gold_graph in gold_graphs:
    pred_edges = set()
    gold_nodes = gold_graph['nodes']
    sentence = gold_graph['sentence']
    dep = dep_predictor.predict(sentence)['predicted_heads']
    dep_to_node = [-1 for _ in range(len(dep))]
    # build gold edges
    for node_num,node in enumerate(gold_nodes):
        for i in range(node['start'],node['end']):
            dep_to_node[i] = node_num
    for ed,st in enumerate(dep):
        if st == 0:
            continue
        node_from = dep_to_node[st-1]
        node_to = dep_to_node[ed]
        if node_from == node_to:
            continue
        word_from = -1 if node_from==-1 else gold_nodes[node_from]['text']
        word_to = -1 if node_to==-1 else gold_nodes[node_to]['text']
        pred_edges.add((word_from, word_to, ''))
    
    gold_edges = set()
    for node_num,node in enumerate(gold_nodes):
        if node['arg1'][0]!=-1:
            gold_edges.add((node_num,node['arg1'][0],'ARG1'))
        if node['arg2'][0]!=-1:
            gold_edges.add((node_num,node['arg2'][0],'ARG2'))
        
    y.append(to_true_edge(gold_nodes,gold_edges))
    y_pred.append(pred_edges)

In [170]:
eval(y,y_pred)

Edge_level, p:26.29695885509839, r:39.729729729729726, f1:31.646932185145317
Graph_level acc: 0.0
